# SIMPLE/HTR <- colab version

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import pandas as pd
from PIL import Image
import numpy as np
import os
import csv
from io import BytesIO

import pickle


"""
IAM Dataset download
"""

if not os.path.exists('/content/data/') :
    os.mkdir("/content/data")

splits = {'train': 'data/train.parquet', 'validation': 'data/validation.parquet', 'test': 'data/test.parquet'}


max_len = -1
str_len = -1
for iter in ['test', 'validation', 'train']:

    if not os.path.exists(f'/content/data/{iter}') :
        os.mkdir(f"/content/data/{iter}")
    df = pd.read_parquet("hf://datasets/Teklia/IAM-line/" + splits[iter])
    df_csv = df['text']

    with open(os.path.join("/content/data",f"{iter}.csv"), 'w', newline="") as csvfile:
        spanwriter = csv.writer(csvfile, delimiter = '\t', quotechar="|")
        spanwriter.writerow(["_path","text"])
        for id, txt in enumerate(df_csv):
            spanwriter.writerow([id,txt])
            if len(txt) > str_len:
                str_len =len(txt)

    #df_csv.to_csv(path_or_buf=os.path.join('/content/data', f'{iter}.csv'), sep='\t')

    df_imag = df['image']
    for idx, img in enumerate(df_imag):
        image = Image.open( BytesIO(img['bytes']))

        image.save(os.path.join(f"./data/{iter}",f"{idx}.jpg" ))

        width_size , height_size = image.size
        if(width_size > max_len):
            max_len =width_size

        if(height_size != 128):
            print(height_size)

print(max_len)
print(str_len)

#pre-processing
retain aspect ratio of images and use batches of padded images in order to effectively use mini-batch Stochastic Gradient Descent(SGD)


In [ ]:
# All images are resized to a resulution of 128x1024 pixels for line images or 64x256 pixels.
# Initial images are padded in order to attain the aformentioned fixed size.

# IAM dataset image have a size width x 128(height), the max width size = 5027
import torch
import math
from torch import nn
from PIL import Image
import numpy as np
from torchvision.transforms import functional as F

# target_size : 1024 x 128
def preprocess(image) :
    max_height = 128
    max_width = 1024
    #image = image.convert("L")

    width_size, height_size = image.size
    if (height_size > 128):
        print(width_size, height_size)
        ratio = 128/height_size
        #image = image.resize((math.ceil(width_size*ratio), 128))
        image = F.resize(image , ( 128,math.ceil(width_size*ratio)))
        width_size, height_size = image.size

    if (width_size > 1024):
        ratio = 1024/width_size
        #image = image.resize( (1024, math.ceil(height_size *ratio) ) )
        image =F.resize(image , (math.ceil(height_size *ratio) ,1024))
    image = np.array(image)
    image = torch.tensor(image)
    image = padd_img(image)

    return image


    # padd it 1024x128
def padd_img(img):
    height_tar, width_tar = 128,1024
    height_size, width_size = img.shape

    ## 128 - 51 = 77
    ## 77/2 = > 38  76 + 51 =127

    left = math.floor((width_tar - width_size)/2)
    right =math.floor((width_tar -width_size)/2)
    if(width_size %2 == 1):
        right = math.floor((width_tar - width_size)/2) +1
    top = math.floor((height_tar - height_size)/2)
    bottom = math.floor((height_tar - height_size)/2)
    if (height_size %2 ==1):
        bottom = math.floor((height_tar - height_size)/2) + 1

    img = nn.functional.pad(img,(left,right,top,bottom),mode="constant", value=255)
    return img


#During traing, image augmentation is performed.
#considering only rotation and skew of small magnitude in order to generate valid images
#Additionaly, gaussian nosie is added to the images
from torchvision.transforms import v2
"""
transforms = v2.RandomApply(
    torch.nn.ModuleList([
        v2.RandomAffine(degrees=(-10,10),translate=(0.01,0.05), scale = (0.8,1.2),shear = (-10,10))
        ]) , p=0.5 )
"""
transforms = v2.RandomApply(
    torch.nn.ModuleList([
        v2.RandomAffine(degrees = (0,0),translate=(0.01,0.05), scale = (0.8,1),shear = (-10,10),fill=255)
        ]) , p=0.5 )

# rotation image ... are cutted so did;nt show the character in image ...
# even i didn't see the chacracter it is... not good
# rotation, translation, scaling and shearing  and gray-scale erosion and dilation
# the paper named "Are Multidimensional Recurrent Layers Really Necessary for Handwritten Text Recognition?" said it toolkit defualt value but didn't found it

"""
We perform adequate random distortions on the input images,
in order to artificially augment the training samples and reduce overfitting.

These distortions include: rotation,translation, scaling and shearing
(all performed as a single affine transform) and gray-scale erosion and dilation.
Each of these operations is applied dynamically and independently on each image of the training batch (each with 0.5 probability).
Thus, the exact same image is virtually never observed twice
during training.
The parameters controlling each distortion (e.g. rotation angle, scaling factor, erosion kernel, etc.) are sampled from a fixed distribution.
    -- Are Multidimensional Recurrent Layers Really Necessary for Handwritten Text Recognition?
"""
# transforms  => preprocessing


#Each word/line transcription has spaces added before and after, "He rose from" => " He rose from "

def pre_processing_target(label):
    return " " + label + " "
target_transforms = pre_processing_target

#This operation aims to assist the system during the training phase, For the testing phase, these additional spaces are discarded


In [ ]:
import torch
from torchvision.transforms import functional as F
import os
from PIL import Image
import pandas as pd

class IAMData():
    def __init__(self,split ,root_path = '/content/data' , transform =None,target_transform = None ):

        assert os.path.exists(root_path), "valueError: IAMData 'wrong root_path' "

        assert  split in ['train', 'test', 'validation'] , "valueError: IAMData 'split must be one of ['train', 'test', 'validation]'"

        self.preprocess = None
        if split == 'train':
            self.preprocess = preprocess
            char_set = set()


        self.annotations_file = os.path.join(root_path, f"{split}.csv")
        self.img_dir = os.path.join(root_path, split)

        self.transform = transform
        self.target_transform = target_transform

        self.img_labels = pd.read_csv(self.annotations_file, delimiter = '\t', quotechar='|')


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir, f"{self.img_labels.iloc[idx,0]}.jpg")

        image = Image.open(img_path)
        label =self.img_labels.iloc[idx,1]

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        if self.preprocess :
            image = self.preprocess(image)

        return image, label


In [ ]:
test_ds = IAMData('test')
print(test_ds[3])

In [ ]:
train_dst = IAMData('train', transform= transforms , target_transform= target_transforms)
print(train_dst[9])



In [ ]:
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

for i in range(5):
    img, label = train_dst[i]

    plt.figure()
    plt.imshow(to_pil_image(img), cmap = 'gray')
    print(label)
    print(img.shape)

#architectural
replace the column-wise concatenation step between the CNN backbone and the recurrent head with a max-pooling step. such a choice not only redueces the required parameters but has an intuitive motivation: we care only about the existence of a character and not its vertical poision

#Convolutional Backbone

In [ ]:
"""
In our model, the convolutional backbone is made
up of standard convolutional layers and ResNet blocks [12], interspersed with
max-pooling and dropout layers.
"""

"""class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

"""
import torch
from torch.nn import functional as F

class Residual_Block(nn.Module):
    def __init__(self,in_channels, out_channels,stride =1):
        super(Residual_Block,self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

class ConvolutionalBackbone(nn.Module):
    def __init__(self):
        super(ConvolutionalBackbone, self).__init__()

        self.CNNLayer = nn.Conv2d(1,32,(7,7), stride=1 , padding=3)

        self.MaxPoolFirst = nn.MaxPool2d((2,2))
        self.ResBlockFirst = nn.Sequential(
              Residual_Block(32,64),
              Residual_Block(64,64)
        )

        self.MaxPoolSecond = nn.MaxPool2d((2,2))
        self.ResBlockSecond = nn.Sequential(
              Residual_Block(64,128),
              Residual_Block(128,128),
              Residual_Block(128,128),
              Residual_Block(128,128)
        )

        self.MaxPoolThird = nn.MaxPool2d((2,2))
        self.ResBlockThird = nn.Sequential(
              Residual_Block(128,256),
              Residual_Block(256,256),
              Residual_Block(256,256),
              Residual_Block(256,256)
        )

    def forward(self,x):
        x = x.reshape([-1,1,128,1024])
        #[128,1024]
        logits = self.CNNLayer(x)
        logits = self.MaxPoolFirst(logits)
        logits = self.ResBlockFirst(logits)

        logits = self.MaxPoolSecond(logits)
        logits = self.ResBlockSecond(logits)

        logits = self.MaxPoolThird(logits)
        logits = self.ResBlockThird(logits)

        #[batch, channel, height, width ] =[batch, 256,16,128]
        return logits




# the first layer is a 7x7 convolution with 32 output channels


#

In [ ]:
model = ConvolutionalBackbone()

test_ds = IAMData('test')
a = torch.rand(2,1,128,1024)
a = model(a)
print(a.shape)

#Flattening Operation => ColumneWiseMaxPooling

In [ ]:
"""
    The convolutional backbone output should be transformed into a sequence of features in order to processed by recurrent networks.
    Typical HTR approaches, assume a column-wise approach (towards the writing direction) to ideally simulate a character by character processing.
    the CNN output is flattened by a max-pooling operation in a column-wise manner

    ... Apart from the apparent computational advantage,
    column-wise max-pooling achieves model translation invariance in the vertical direction
"""

#.. it is too small, it is better to apply it just function ... in whole class ...
class ColumnwiseMaxPool(nn.Module):
    def __init__(self):
        super(ColumnwiseMaxPool,self).__init__()

    def forward(self,CNNoutput):
        MaxPoolData  =torch.max(CNNoutput, dim = 2)
        #[batch,channel,width]
        return MaxPoolData

#Recurrent Head


In [ ]:
"""
    The recurrent component consists of 3 stacked Bidirectional Long Short-Term Memory (BiLSTM) units of hidden size 256. These are followed
by a linear projection layer, which converts the sequence to a size equal to the number of possible character tokens, nclasses (including the blank character, required by CTC).

    The final output of the recurrent part can be translated into a sequence of probability distributions by applying a softmax operation.
"""

"""
 During evaluation, the aforementioned greedy decoding is performed by selecting the character
  with the highest probability at each step and then removing the blank characters from the resulting sequence [8].
"""

class RecurrentHead(nn.Module):
    def __init__(self, nclasses):
        super(RecurrentHead,self).__init__()
        self.BiLSTM = nn.LSTM(256,256,num_layers=3,bidirectional=True)
        self.Projection = nn.Linear(512,nclasses)
        #nclasses include the blank character

    def forward(self,x):
        x = x.permute([2,0,1])
        #[batch,channel, width] = > [width,batch,channel]
        outputs, (hidden , cell) = self.BiLSTM(x)
        logits = self.Projection(outputs)
        #[width,batch,2 * channel] => [width,batch, nclasses ]
        logits = logits.transpose(0,1)
        #[batch,width,nclasses]

        return logits



# CTC shortcut


In [ ]:
"""
Architecture-wise, the CTC shortcut module consists only of
a single 1D convolutional layer, with kernel size 3. Its output channels equal
to the number of the possible character tokens (nclasses). Therefore, the 1D
convolutional layer is responsible for straightforwardly encoding context-wise
information and providing an alternative decoding path.
"""

class CTCshortcut(nn.Module):
    def __init__(self, nclasses):
        # nclasses include the black_id
        super(CTCshortcut,self).__init__()

        self.ConLayer = nn.Conv1d(128,nclasses,3,padding = 1)


    def forward(self,x):
        logits = self.ConLayer(x)
        #[batch,channel,width]=>[batch,nclasses,width]
        logits = logits.transpose(1,2)
        #[batch,nclasses,width] => [batch,width,nclasses]

        return logits


#CTC Loss

the multi-task loss is written as

$L_{CTC}(f_{rec}(f_{cnn}(I)) ;s ) +0.1 L_{CTC}((f_{shortcut}(f_{cnn}(I)); s))$


"""

    The CTC shortcut is trained along with the main architecture using
    a multitask loss by adding the corresponding CTC losses of the two branches with the appropriate weights

    Since CTC shortcut acts only as an auxiliary training path, it is weighted by 0.1
    to reduce its relative contribution to the overall loss.

"""


In [ ]:
#total HTRnet

class HTRnet(nn.Module):
    def __init__(self, nclasses):
        #nclasses include the blank_id
        super(HTRnet,self).__init__()

        self.CNNBacbone = ConvolutionalBackbone()
        # columnewise Maxpooling

        self.RecHead = RecurrentHead(nclasses)
        self.CTCshort = CTCshortcut(nclasses)


    def forward(self, x):
        features = self.CNNBackbone(x)
        #[batch,channel,height, width]

        # columnewise Maxpooling
        colMaxPool = torch.max(features,dim =2)
        #[batch,channel, width]

        RecOutput =  self.RecHead(colMaxPool)
        CTCOutput = self.CTCshort(colMaxPool)

        return RecOutput, CTCOutput


In [ ]:
model = HTRnet(93)
print(model)

# training
add and extra shortcut branch, consisting of a single 1D convoluion layer, at the ouptput of the CNN backbone. this branch results to a an extra character sequnce estimation, trained in parallel to the recurrent branch. both branches use a CTC loss. the motivation behind such a choice comes from the increased difficulty of training recurrent layers. However, if such a strainghtforward shortcut exists, the output of the CNN backbone should coverge to more discriminative features, ideal for fully harnessing the power of recurrent layers compared to an end-to-end training scheme

In [ ]:
class HTRTrainer(nn.Module):
    def __init__(self):
        super(HTRTrainer,self).__init__()

        self.prepare_dataloaders()
        self.prepare_net()
        self.prepare_losses()
        self.prepare_optimizers()

    def prepare_dataloaders(self):
        train_ds = IAMData('train',transform = transforms,target_transform= target_transforms)



